# Deep Reinforcement Learning

 Reinforcement Learning (RL) is an approach wherein an agent learns to make sequential decisions by interacting with an environment. The objective is for the agent to maximize the cumulative reward it receives over time.
 The agent goes through this process by repeatedly evaluating the consequences of its actions, trying to select actions that lead to better outcomes.

To do this, we will use Gym, an platform for developing and comparing reinforcement learning algorithms. Gym provides an interface for interacting with different environments, it accepts actions from agents and plays them out in an environment, providing rewards.


## Environment

We will be using `CartPole` environment from gym's library for this assignment.  In this environment, a pole is attached by an un-actuated joint to a cart, which moves along a frictionless track. The pendulum is placed upright on the cart and the goal is to balance the pole by applying forces in the left and right direction on the cart.

You can use the code below to run an instance of a random agent in this environment and see the results.

In [ ]:
from IPython.display import HTML
from base64 import b64encode

def show_video(path):
    mp4 = open(path, 'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    return HTML("""
    <video width=400 controls>
          <source src="%s" type="video/mp4">
    </video>
    """ % data_url)

In [ ]:
!pip install gym[atari,accept-rom-license] -qq
!pip install imageio -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 37.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import cv2
import gym
import imageio
import numpy as np
from gym import spaces

We use `gym.make()` to make an instance of a certain environemtn. We can then use `.step()` method which accepts an action as input and performs it. Before that we reset the environment to its initial state by using `.reset()` method.

In [ ]:
env_name = 'CartPole-v1'

env = gym.make(env_name)

env.reset()

frames = []

for _ in range(500):
    action = env.action_space.sample()

    obs, reward, done, _ = env.step(action)

    frames.append(env.render(mode='rgb_array'))

    if done:
        env.reset()

env.close()
imageio.mimsave('./cartpole.mp4', frames, fps=25)

In [ ]:
show_video('./cartpole.mp4')

As you can see, the cart fails to keep the balance of the pole. In the next section we will train an agent to learn how to perform this task.

## Algorithm
We will be using A2C algorithm.

Advantage Actor-Critic (A2C) is a reinforcement learning algorithm.
It consists of an actor (which predicts the best action based on the current state) and a critic (which estimates the state's value function to measure expected future rewards).

We will implement this together step by step.




In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions.categorical import Categorical

import numpy as np
import gym
from collections import deque
from tqdm import tqdm

## Neural Network

Here we design a simple feed forward model to embed the observation from the environment to a hidden layer. We then use two fully connected layers on top of the hidden layer, to predict the next action and estimate the value of current state. This acts as both actor, and critic.


In [ ]:
class ActorCritic(nn.Module):
    def __init__(self, hidden_size, num_inputs, num_outputs):
        super(ActorCritic, self).__init__()
        self.fc1 = nn.Linear(num_inputs, hidden_size)
        self.fc2_actor = nn.Linear(hidden_size, num_outputs)
        self.fc2_critic = nn.Linear(hidden_size, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        action_probs = F.softmax(self.fc2_actor(x), dim=-1)
        value = self.fc2_critic(x)
        return action_probs, value

## A2C

The A2C algorithm aims to jointly train both the actor and the critic to improve the policy. It does this by updating the parameters
of the actor to increase the likelihood of good actions and updating the parameters
of the critic to better estimate the value function.

In each iteration A2C plays the until it ends. During this time it records log probabality of actions, rewards, and predicted values in each step. These values will be used to update the model at the end of this trajectory.

The actor is updated using the objective below:

$$ L_{\text{actor}} = -\log \pi(a|s;\theta) \times A(s, a) $$
Where advantage is calculated as:
$$A(s, a) = Q(s, a) - V(s) $$

Namely the function $Q(s,a)$ is the estimated value of taking action
$a$
 in state
$s$.
$V(s)$ is the predicted value of our critic.

This loss function aims to improve the probability of playing actions that result in higher rewards.

As for the critic the loss function is defined as a simple mean square loss between actual value of an state and the predicted one:

$$ L_{\text{critic}} = \frac{1}{2} ( R - V(s))^2 $$

In [ ]:
class A2CAgent:
    def __init__(self, env, num_episodes=1000, max_steps=500, gamma=0.99, lr=1e-3, hidden_size=256):
        self.env = env
        self.num_episodes = num_episodes
        self.max_steps = max_steps
        self.gamma = gamma
        self.lr = lr
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        num_inputs = env.observation_space.shape[0]
        num_outputs = env.action_space.n

        self.policy_net = ActorCritic(hidden_size, num_inputs, num_outputs).to(self.device)
        self.optimizer = optim.Adam(self.policy_net.parameters(), lr=self.lr)

    def choose_action(self, state):
        state = torch.FloatTensor(state).to(self.device)
        action_probs, _ = self.policy_net(state)
        dist = Categorical(action_probs)
        action = dist.sample()
        return action.item()

    def compute_returns(self, rewards):
        returns = []
        discounted_reward = 0
        for r in reversed(rewards):
            discounted_reward = r + self.gamma * discounted_reward
            returns.insert(0, discounted_reward)
        returns = torch.tensor(returns).to(self.device)
        return returns

    def train(self):
        episode_rewards = []
        for episode in tqdm(range(self.num_episodes)):
            state = self.env.reset()
            log_probs = []
            values = []
            rewards = []
            episode_reward = 0

            for step in range(self.max_steps):
                state = torch.FloatTensor(state).to(self.device)
                action_probs, value = self.policy_net(state)
                dist = Categorical(action_probs)
                action = dist.sample()
                next_state, reward, done, _ = self.env.step(action.cpu().numpy())

                log_prob = dist.log_prob(action).unsqueeze(0)
                log_probs.append(log_prob)
                values.append(value.detach())
                rewards.append(reward)
                episode_reward += reward

                state = next_state

                if done:
                    break

            log_probs = torch.cat(log_probs, dim=0)
            values = torch.cat(values)

            returns = self.compute_returns(rewards)

            advantages = returns - values

            actor_loss = -(advantages.detach() * log_probs).mean()

            critic_loss = F.mse_loss(values, returns)

            loss = actor_loss + critic_loss

            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()

            episode_rewards.append(episode_reward)

            if (episode + 1) % 10 == 0:
                print(f"Episode {episode + 1}, Reward: {episode_reward}")

        self.env.close()
        return episode_rewards


Define the model and set hyperparameters.

In [ ]:
env_name = 'CartPole-v1'
num_episodes = 1300
max_steps = 500
lr = 0.001
hidden_size = 256

env = gym.make(env_name)

a2c_model = A2CAgent(env, num_episodes=num_episodes, max_steps=max_steps, lr=lr, hidden_size=hidden_size)

Train the model.

In [ ]:
rewards = a2c_model.train()

  1%|          | 13/1300 [00:00<00:46, 27.92it/s]

Episode 10, Reward: 11.0


  2%|▏         | 22/1300 [00:00<00:57, 22.13it/s]

Episode 20, Reward: 34.0


  3%|▎         | 34/1300 [00:01<00:51, 24.35it/s]

...

Episode 1280, Reward: 357.0


 99%|█████████▉| 1290/1300 [07:41<00:07,  1.37it/s]

Episode 1290, Reward: 251.0


100%|██████████| 1300/1300 [07:49<00:00,  2.77it/s]

Episode 1300, Reward: 499.0


## Evaluation

Use the `choose_action` method of the trained agent to evaluate its performance.

In [ ]:
env = gym.make(env_name)
model = a2c_model

num_episodes = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

frames = []
episode_rewards = []


for i in range(num_episodes):
    state = env.reset()
    episode_reward = 0
    done = False

    while not done:
        frame = env.render(mode='rgb_array')
        frames.append(frame)

        action = model.choose_action(state)

        next_state, reward, done, _ = env.step(action)

        state = next_state
        episode_reward += reward

    episode_rewards.append(episode_reward)
    print(f"Episode {i+1} Reward: {episode_reward}")

env.close()

episode_rewards = np.array(episode_rewards)
avg_reward = np.mean(episode_rewards)
print(f"Average Reward over {num_episodes} episodes: {avg_reward}")

output_path = './test.mp4'
imageio.mimsave(output_path, frames, fps=25)

Episode 1 Reward: 320.0
Episode 2 Reward: 500.0
Episode 3 Reward: 500.0
Episode 4 Reward: 500.0
Episode 5 Reward: 237.0
Episode 6 Reward: 500.0
Episode 7 Reward: 500.0
Episode 8 Reward: 500.0
Episode 9 Reward: 500.0


Episode 10 Reward: 500.0
Average Reward over 10 episodes: 455.7


In [ ]:
show_video('./test.mp4')